# Concatenate chain sequences into one structure in fasta

In [2]:
import pandas as pd
import json
import re
import matplotlib.pyplot as plt
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [231]:
data = pd.read_json("data/20240827_data.json")

In [232]:
data

,pdb,picture_path,chains,authors,classification,classification_suggested,classification_suggested_reason,subtitle,tags,keywords,...,publication_country,abstract,crystal_structure,symmetry,exptl_method,formula_weight,synthesis_comment,review,previous_design,next_design
0,1abz,https://cdn.rcsb.org/images/structures/1abz_as...,"[{'chain_id': 'A', 'chain_source': 'unknown', ...","[{'forename': 'Y.', 'surname': 'Fezoui'}, {'fo...",unknown,[],[],"Alpha-t-alpha, a de novo designed peptide, nmr...","[helix-turn-helix, de novo design]","[derived, 20, angle, restraints, structures, n...",...,US,alpha t alpha is a 38-residue peptide designed...,"{'length_a': '1.000', 'length_b': '1.000', 'le...",P 1,[SOLUTION NMR],4446.004,,1,8y33,1al1
1,1al1,https://cdn.rcsb.org/images/structures/1al1_as...,"[{'chain_id': 'A', 'chain_source': 'unknown', ...","[{'forename': 'C.P.', 'surname': 'Hill'}, {'fo...",unknown,"[computational, consensus]",[Dek's classification: original de novo design...,Crystal structure of alpha1: implications for ...,[synthetic protein model],"[bundles, angles, designed, three, leucine, bu...",...,US,X-ray diffraction shows the structure of a syn...,"{'length_a': '62.350', 'length_b': '62.350', '...",I 41 3 2,[X-RAY DIFFRACTION],1441.775,The peptide was chemically synthesized,1,1abz,1bb1
2,1bb1,https://cdn.rcsb.org/images/structures/1bb1_as...,"[{'chain_id': 'A', 'chain_source': 'synthetic ...","[{'forename': 'S.', 'surname': 'Nautiyal'}, {'...",unknown,"[rational, computational, physics-based]",[Dek's classification: original de novo design...,"Crystal structure of a designed, thermostable ...","[de novo protein design, coiled coil]","[assembly, map, charged, distinct, arrangement...",...,US,Electrostatic interactions are often critical ...,"{'length_a': '21.930', 'length_b': '35.010', '...",P 1 21 1,[X-RAY DIFFRACTION],3703.303,,1,1al1,1byz
3,1byz,https://cdn.rcsb.org/images/structures/1byz_as...,"[{'chain_id': 'A,B,C,D', 'chain_source': 'unkn...","[{'forename': 'G.G.', 'surname': 'Prive'}, {'f...",unknown,[],[Dek's classification: relative of another de ...,"Designed peptide alpha-1, p1 form","[de novo protein, helical bilayer; biomaterial]","[yields, opposing, charged, distinct, packed, ...",...,US,A 12-residue peptide designed to form an alpha...,"{'length_a': '20.846', 'length_b': '20.909', '...",P 1,[X-RAY DIFFRACTION],1441.775,,1,1bb1,1coi
4,1coi,https://cdn.rcsb.org/images/structures/1coi_as...,"[{'chain_id': 'A', 'chain_source': 'synthetic ...","[{'forename': 'N.L.', 'surname': 'Ogihara'}, {...",unknown,[],[],Designed trimeric coiled coil-vald,"[alpha-helical bundle, protein design, coiled ...","[vqalekk, data, bundles, among, mainly, design...",...,US,The three-dimensional structure of the 29-resi...,"{'length_a': '33.600', 'length_b': '33.600', '...",P 3 2 1,[X-RAY DIFFRACTION],3206.705,,1,1byz,1cos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,8vt8,https://cdn.rcsb.org/images/structures/8vt8_as...,"[{'chain_id': 'A,B,C,D', 'chain_source': 'unkn...","[{'forename': 'P.', 'surname': 'Ganatra'}, {'f...",unknown,[],[],Uic-13-bif-a4dab nbd-cl binding,"[de novo protein, synthetic construct]","[porous, assembly, conjugated, strongly, corre...",...,US,Peptide-based frameworks aim to integrate prot...,"{'length_a': '23.753', 'length_b': '18.836', '...",P 1 21 1,[X-RAY DIFFRACTION],1657.955,,1,8vsf,8vw7
1446,8vw7,https://cdn.rcsb.org/images/structures/8vw7_as...,"[{'chain_id': 'A,B', 'chain_source': 'unknown'...","[{'forename': 'P.', 'surname': 'Ganatra'}, {'f...",unknown,[],[],Uic-11-bpe-agno3 silver binding with uic-11-bpe,"[de novo protein, synthetic construct]","[porous, assembly, conjugated, strongly, corre...",...,US,Peptide-based frameworks aim to integrate prot...,"{'length_a': '9.025', 'length_b': '30.416', 'l...",P 1,[X-RAY DIFFRACTION],1530.772,,1,8vt8,8vw8
1447,8vw8,https://cdn.rcsb.org/images/structures/8vw8_as...,"[{'chain_id': 'A', 'chain_source': 'unknown', ...",

# Download fasta files from [here](https://www.rcsb.org/downloads/fasta)

In [233]:
fasta_codes_list_str = [""] * ((len(data) // 1000) + 1)

for i, row in data.iterrows():
    index = i // 1000  # This determines the batch index
    if not fasta_codes_list_str[index]:
        fasta_codes_list_str[index] = row["pdb"]
    else:
        fasta_codes_list_str[index] += ", " + row["pdb"]

In [234]:
fasta_codes_list_str[1]

'7q1t, 7q3j, 7q3k, 7q58, 7qdi, 7qdj, 7qdk, 7qnl, 7qnp, 7qsv, 7qsw, 7qsx, 7qsy, 7qsz, 7qt1, 7qvi, 7qwa, 7qwb, 7qwc, 7qwd, 7qwe, 7qxj, 7r0r, 7r8y, 7r8z, 7rkc, 7rmx, 7rmy, 7rx5, 7s5b, 7skn, 7sko, 7skp, 7smj, 7sq3, 7sq4, 7sq5, 7t03, 7t2y, 7t6e, 7tal, 7tbn, 7tbo, 7tbq, 7tcd, 7tjl, 7tls, 7tlu, 7tm1, 7tm2, 7tma, 7tme, 7tmh, 7tmi, 7tmj, 7tmk, 7tml, 7ubc, 7ubd, 7ube, 7ubf, 7ubg, 7ubh, 7ubi, 7ucp, 7udj, 7udk, 7udl, 7udm, 7udn, 7udo, 7udv, 7udw, 7udx, 7udy, 7udz, 7ue2, 7uek, 7uit, 7ujj, 7ukb, 7unh, 7uni, 7unj, 7upo, 7upp, 7upq, 7uqa, 7ur7, 7ur8, 7uuq, 7uwy, 7uwz, 7uzl, 7vql, 7vqv, 7vqw, 7vty, 7vu4, 7x7i, 7x7j, 7x7k, 7xm1, 7xyq, 7y9c, 7yh8, 7z71, 7z72, 7z73, 7z7c, 7z7e, 7zbs, 7zcj, 7zk1, 7zp5, 7zp6, 7zp7, 7zph, 7zpw, 7zpz, 7zq2, 7zq8, 7zqa, 7zqg, 7zss, 7zyu, 8a09, 8a19, 8a1a, 8a3g, 8a3i, 8a3j, 8a3k, 8a50, 8a51, 8a85, 8adx, 8aed, 8ah9, 8aiw, 8ak1, 8ang, 8anh, 8ani, 8ank, 8anm, 8ao0, 8ao1, 8apy, 8b15, 8b16, 8b45, 8bcs, 8bct, 8bfd, 8bfe, 8bl5, 8bl6, 8bl9, 8bu0, 8c3e, 8c3w, 8c66, 8ch0, 8chy, 8cig, 8ck

# Load fast files

In [237]:
records = SeqIO.to_dict(SeqIO.parse("./data/designed_sequences_1450.fasta", "fasta"))

In [239]:
records

{'1ABZ_1|Chain': SeqRecord(seq=Seq('XDWLKARVEQELQALEARGTDSNAELRAMEAKLKAEIQKX'), id='1ABZ_1|Chain', name='1ABZ_1|Chain', description='1ABZ_1|Chain A|ALPHA-T-ALPHA|', dbxrefs=[]),
 '1AL1_1|Chain': SeqRecord(seq=Seq('XELLKKLLEELKG'), id='1AL1_1|Chain', name='1AL1_1|Chain', description='1AL1_1|Chain A|ALPHA HELIX PEPTIDE: ELLKKLLEELKG|null (32630)', dbxrefs=[]),
 '1BB1_1|Chain': SeqRecord(seq=Seq('XAEIAAIEYEQAAIKEEIAAIKDKIAAIKEYIAAIX'), id='1BB1_1|Chain', name='1BB1_1|Chain', description='1BB1_1|Chain A|DESIGNED, THERMOSTABLE HETEROTRIMERIC COILED COIL|synthetic construct (32630)', dbxrefs=[]),
 '1BB1_3|Chain': SeqRecord(seq=Seq('XAEIAAIKYKQAAIKNEIAAIKQEIAAIEQMIAAIX'), id='1BB1_3|Chain', name='1BB1_3|Chain', description='1BB1_3|Chain C|DESIGNED, THERMOSTABLE HETEROTRIMERIC COILED COIL|synthetic construct (32630)', dbxrefs=[]),
 '1BB1_2|Chain': SeqRecord(seq=Seq('XEKIAAIKEEQAAIEEEIQAIKEEIAAIKYLIAQIX'), id='1BB1_2|Chain', name='1BB1_2|Chain', description='1BB1_2|Chain B|DESIGNED, THERMOSTABL

# Group only designed sequences per pdb

In [240]:
def extract_chain_labels(fasta_description):
    chain_part = (fasta_description.split("|")[1]).replace("Chains ", "").replace("Chain ", "").strip()
    auth_chain_ids = []
    pdb_chain_ids = []

    all_possible_chain_ids_pattern = re.compile(r'\b([A-Za-z0-9]+)\b(?![^\[]*\[auth\])')
    all_possible_chain_ids = all_possible_chain_ids_pattern.findall(chain_part)

    auth_idx = [idx for idx, element in enumerate(all_possible_chain_ids) if element == "auth"]
    for i in auth_idx:
        auth_chain_ids.append(all_possible_chain_ids[i+1])

    i = 0
    while i < len(all_possible_chain_ids):
        if i in auth_idx:
            i += 2
            continue
        pdb_chain_ids.append(all_possible_chain_ids[i])
        i += 1
    
    # Return the list of matches
    return pdb_chain_ids, auth_chain_ids

In [241]:
grouped_sequences = {}

# Iterate through the records and group them by PDB code
for record_id, record in records.items():
    pdb_code = record_id[:4].lower()  # Extract the first four characters (PDB code)
    pdb_chain_ids, auth_chain_ids = extract_chain_labels(record.description)

    match_found = False
    if data["pdb"].str.contains(pdb_code).any():

        if "32630" in record.description.split("|")[-1]:
            if pdb_code not in grouped_sequences:
                grouped_sequences[pdb_code] = []
            grouped_sequences[pdb_code].append(str(record.seq))
            continue
            
        chain_array = data[data["pdb"] == pdb_code]["chains"].values
        list_of_chains = [item for sublist in chain_array for item in sublist]
            
        for chain in list_of_chains:
            if chain["chain_type"] in ["D", "U", "M"]:
                chain_id = chain["chain_id"].split(",")
                if any(label in pdb_chain_ids for label in chain_id) or any(label in auth_chain_ids for label in chain_id):
                    if pdb_code not in grouped_sequences:
                        grouped_sequences[pdb_code] = []
                    grouped_sequences[pdb_code].append(str(record.seq))
                    match_found = True  # Set flag to True if a match is found
                    break  # Exit the inner loop since a match is found
        if match_found:
            continue

In [242]:
grouped_sequences

{'1abz': ['XDWLKARVEQELQALEARGTDSNAELRAMEAKLKAEIQKX'],
 '1al1': ['XELLKKLLEELKG'],
 '1bb1': ['XAEIAAIEYEQAAIKEEIAAIKDKIAAIKEYIAAIX',
  'XAEIAAIKYKQAAIKNEIAAIKQEIAAIEQMIAAIX',
  'XEKIAAIKEEQAAIEEEIQAIKEEIAAIKYLIAQIX'],
 '1byz': ['XELLKKLLEELKG'],
 '1coi': ['XEVEALEKKVAALESKVQALEKKVEALEHGX'],
 '1cos': ['XEWEALEKKLAALESKLQALEKKLEALEHGX'],
 '1d7t': ['GCPYNPKC'],
 '1djf': ['QAPAYKKAAKKLAES'],
 '1e0m': ['SMGLPPGWDEYKTHNGKTYYYNHNTKTSTWTDPRMSS'],
 '1ec5': ['XDYLRELLKLELQLIKQYREALEYVKLPVLAKILEDEEKHIEWLETILGX'],
 '1fme': ['EQYTAKYKGRTFRNEKELRDFIEKFKGR'],
 '1fmh': ['XEVAQLEKEVAQAEAENYQLEQEVAQLEHECGX',
  'XEVQALKKRVQALKARNYAAKQKVQALRHKCGX'],
 '1fsd': ['QQYTAKIKGRTFRNEKELRDFIEKFKGR'],
 '1fsv': ['QQYTAKIKGRTFRNEKELRDFIEKFKGR'],
 '1g6u': ['SLAALKSELQALKKEGFSPEELAALESELQALEKKLAALKSKLQALKG'],
 '1hcw': ['XYTVPSATFSRSDELAKLLRLHAGX'],
 '1hqj': ['DELERRIRELEARIK'],
 '1ic9': ['SKYEYTIPSYTFRGPGCPTLKPAITVRCE'],
 '1icl': ['SKYEYTVPSYTFRGPGCPTVKPAISLRCE'],
 '1ico': ['SKYEYTIPSYTFRGPGCPTVKPAVTIRCE'],
 '1j4m': ['

In [243]:
# Concatenate sequences in each group
concatenated_records = []
for pdb_code, sequences in grouped_sequences.items():
    concatenated_seq = ''.join(sequences)  # Concatenate all sequences in the group
    concatenated_record = SeqRecord(Seq(concatenated_seq), id=pdb_code, description=f"Concatenated sequences for {pdb_code}")
    concatenated_records.append(concatenated_record)

In [245]:
unique_ids = set(record[:4].lower() for record in records)
len(unique_ids)

1450

In [246]:
unique_concat_ids = set(c_record.id[:4].lower() for c_record in concatenated_records)
len(unique_concat_ids)

1450

In [248]:
unique_ids.difference(unique_concat_ids)

set()

In [249]:
with open("data/concatenated_designed_sequences_1450.fasta", "w") as output_handle:
    SeqIO.write(concatenated_records, output_handle, "fasta")

# Leave only designed chains in fasta file

In [220]:
records_copy_synth = SeqIO.to_dict(SeqIO.parse("./data/chain_designed_sequences_1450.fasta", "fasta"))

In [221]:
designed_single_sequences = {}

# Iterate through the records and group them by PDB code
for record_id, record in records.items():
    pdb_code = record_id[:4].lower()  # Extract the first four characters (PDB code)
    pdb_chain_ids, auth_chain_ids = extract_chain_labels(record.description)

    match_found = False
    if data["pdb"].str.contains(pdb_code).any():
        if "32630" in record.description.split("|")[-1]:
            continue
        
        chain_array = data[data["pdb"] == pdb_code]["chains"].values
        list_of_chains = [item for sublist in chain_array for item in sublist]
            
        for chain in list_of_chains:
            if chain["chain_type"] == "N":
                chain_id = chain["chain_id"].split(",")
                data_pdb_chain_id = chain["chain_id_pdb"]
                data_auth_chain_id = chain["chain_id_auth"]
                if any(label in pdb_chain_ids for label in chain_id): # any(label in pdb_chain_ids for label in data_pdb_chain_id) or any(label in auth_chain_ids for label in data_auth_chain_id):
                    records_copy_synth.pop(record_id, None)
                    match_found = True  # Set flag to True if a match is found
                    break  # Exit the inner loop since a match is found
        if match_found:
            continue

In [222]:
len(records_copy_synth)

1592

In [217]:
unique_ids_synth = set(record[:4].lower() for record in records_copy_synth)
len(unique_ids_synth)

1450

In [228]:
# Concatenate sequences in each group
concatenated_records = []
for pdb_code, sequences in records_copy_synth.items():
    concatenated_seq = ''.join(sequences)  # Concatenate all sequences in the group
    concatenated_record = SeqRecord(Seq(concatenated_seq), id=pdb_code, description=f"designed single sequences for {pdb_code}")
    concatenated_records.append(concatenated_record)

In [229]:
with open("data/designed_single_sequences_1450.fasta", "w") as output_handle:
    SeqIO.write(concatenated_records, output_handle, "fasta")